In [ ]:
# ◉_◉

In [ ]:
# for doc, score in query_response:
#     print("*" * 40)
#     print(doc.dict().keys())
#     # print(doc.from_orm(doc).dict())
#     print("score: %s" % score)
#     print(doc.page_content)
#     print("*" * 40)

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import PromptTemplate
# Set logging for the queries
import logging

In [16]:
# TODO: Clean PDFS, is everything relevant?
query = """What are some key take aways from the agent tuning to get me started? Lets go through it step by step, and help me devise a plan for implementing some of the ideas in python code."""

In [14]:
llm = ChatOpenAI(temperature=0.44,openai_api_key=oai_sk)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db_conn.as_retriever(), llm=llm
)

In [17]:
unique_docs = retriever_from_llm.get_relevant_documents(query=query)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide me with the main insights from the agent tuning process to help me kickstart my implementation? I would appreciate a step-by-step walkthrough and guidance on how to translate these ideas into Python code.', '', '2. In order to begin implementing the agent tuning ideas, could you please highlight the key lessons learned from this process? It would be great if you could break it down into steps and assist me in devising a Python code plan.', '', '3. To get started with implementing the agent tuning concepts, can you share some important takeaways? I would like to understand the process step by step and receive guidance on how to convert these concepts into Python code.']


In [21]:
unique_docs[0].dict().keys()
unique_docs[0].metadata

{'source': 'temp/2310.12823.pdf'}

In [22]:
proompt = """Use the following bits of information and internalize them. Because, you will then be applying the knowledge in a step by step manner to help a research agent in formulating a plan to implement the ideas in python code. Remember to be concise and pragmatic in your answer, and also focus on changing the most specific thing for the most specific reason. Remember, this means a lot to us, experimentation and research are important and you are good at both.

Here is the relevant information:
{env_context}

And this is the current task at hand:
{task_description}

Your answer:"""

big_proompt = PromptTemplate(
    template=proompt, input_variables=["env_context", "task_description"]
)

In [26]:
response = llm.predict(
    text=big_proompt.format_prompt(env_context=unique_docs, task_description=query).text
)

In [7]:
from utils.utils import extract_functions_from_repo
from pathlib import Path
codedir = Path.cwd() / "temprepo"
all_functions = extract_functions_from_repo(codedir)

Total number of .py files: 230
Total number of functions extracted: 487


In [8]:
all_functions

[{'code': 'def doc_to_text(doc):\n    conv = get_conversation_template("llama-2")\n    conv.set_system_message(\n        "You are a helpful, respectful and honest assistant.")\n    conv.append_message(\n        conv.roles[0], "Please provide a detailed step-by-step solution for the following grade school math question. ")\n    conv.append_message(conv.roles[1], "Ok.")\n    for q, a in shots:\n        conv.append_message(conv.roles[0], q)\n        conv.append_message(conv.roles[1], a)\n    conv.append_message(conv.roles[0], f"\\nQuestion: {doc[\'question\']}")\n    conv.append_message(conv.roles[1], None)\n    return conv.get_prompt()\n\n',
  'function_name': 'doc_to_text',
  'filepath': '/home/sonofman/Research/Arxiv2Arxode/temprepo/eval_general/eval_gsm8k_tgi.py'},
 {'code': 'def clean_answer(text):\n    text = text.split("Question:")[0]\n    return text\n\n',
  'function_name': 'clean_answer',
  'filepath': '/home/sonofman/Research/Arxiv2Arxode/temprepo/eval_general/eval_gsm8k_tgi.py

In [1]:
from embeddings import create_embedding_collection, load_and_chunk_code, get_embedding_func
import openai
import langchain
# db_conn = get_db_connection("agentTuning_code_vecdb")

In [3]:
split_code_docs = load_and_chunk_code("./temprepo/")
split_code_docs
code_embedding_dbconn = create_embedding_collection(
    chunked_docs=split_code_docs, embeddings=get_embedding_func(),
    collection_name="agentTuning_code_vecdb"
)

In [14]:
for doc in code_embedding_dbconn.similarity_search("mmlu eval"):
    print(doc.page_content)
# print(code_embedding_dbconn.similarity_search("mmlu eval")[0].page_content)

def eval_subject(
        model,
        tokenizer,
        subject_name,
        test_df,
        k=5,
        dev_df=None,
        few_shot=False,
        save_result_dir=None,
        **kwargs
):
    result = []
    score = []

    cov = generate_few_shot_prompt(
        k, subject_name, dev_df) if few_shot else []
    all_probs = {'prob_A': [], 'prob_B': [], 'prob_C': [], 'prob_D': []}

    for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
        question = format_example(row, include_answer=False)
        cov.append_message(cov.roles[0], question.split("Answer:")[0].rstrip())
        cov.append_message(cov.roles[1], None)
        full_prompt = cov.get_prompt() + " Answer: "
        cov = generate_few_shot_prompt(
            k, subject_name, dev_df) if few_shot else []

        output, input_info = get_logits(tokenizer, model, [full_prompt])
        assert output.shape[0] == 1
        logits = output.flatten()

        softval = torch.nn.functional.softmax(
            